In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
import nltk.data
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import logging
from gensim.models import Word2Vec,KeyedVectors
import pandas as pd
from nltk.stem.porter import PorterStemmer
from stemming.porter2 import stem
import json

In [16]:
# function to parse sentences to words and remove stopwords from sentence
def sentence_to_wordlist( sentence, remove_stopwords=True ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    #text = BeautifulSoup(sentence).get_text()
    #  
    # 2. Remove non-letters
    text = re.sub("[^a-zA-Z]"," ",sentence)
    #
    # 3. Convert words to lower case and split them
    words = text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [3]:
# function to parse text to sentences using tokenizer mentioned above
def text_to_sentences(text,tokenizer,remove_stopwords=True):
    raw_sentences = tokenizer.tokenize(text.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( sentence_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [4]:
with open('en_articles_unique.json') as data_file:
    data = json.load(data_file)

In [6]:
text_list = []
for attribute,value in data.iteritems():
    text_list.append(value)

In [10]:
count_vect = CountVectorizer()
feature_vecs = count_vect.fit_transform(text_list)
feature_vecs.shape

(10852, 80268)

In [12]:
tfidf_transformer = TfidfTransformer()
feature_vecs_tfidf = tfidf_transformer.fit_transform(feature_vecs)
feature_vecs_tfidf.shape

(10852, 80268)

In [17]:
sentences = []
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
for text in text_list:
    sentences += [[stem(str(word)) for word in sentence] \
                        for sentence in [text_to_sentences(text, tokenizer)]]

In [18]:
print len(sentences)

10852


In [59]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 50    # Word vector dimensionality                      
min_word_count = 2   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 4           # Context window size                                                                                    
#downsampling = 1e-5   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)

print "Training model..."
model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=False)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)
#model.save_word2vec_format(model_name,binary=False)

2017-10-22 15:21:24,003 : INFO : collecting all words and their counts
2017-10-22 15:21:24,004 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-22 15:21:24,157 : INFO : PROGRESS: at sentence #10000, processed 196961 words, keeping 177834 word types
2017-10-22 15:21:24,174 : INFO : collected 191650 word types from a corpus of 214377 raw words and 10852 sentences
2017-10-22 15:21:24,175 : INFO : Loading a fresh vocabulary


Training model...


2017-10-22 15:21:25,113 : INFO : min_count=1 retains 191650 unique words (100% of original 191650, drops 0)
2017-10-22 15:21:25,114 : INFO : min_count=1 leaves 214377 word corpus (100% of original 214377, drops 0)
2017-10-22 15:21:25,770 : INFO : deleting the raw counts dictionary of 191650 items
2017-10-22 15:21:25,776 : INFO : sample=1e-05 downsamples 353 most-common words
2017-10-22 15:21:25,778 : INFO : downsampling leaves estimated 211843 word corpus (98.8% of prior 214377)
2017-10-22 15:21:25,779 : INFO : estimated required memory for 191650 words and 100 dimensions: 249145000 bytes
2017-10-22 15:21:26,405 : INFO : resetting layer weights
2017-10-22 15:21:28,901 : INFO : training model with 10 workers on 191650 vocabulary and 100 features, using sg=0 hs=0 sample=1e-05 negative=5 window=10
2017-10-22 15:21:28,902 : INFO : expecting 10852 sentences, matching count from corpus used for vocabulary survey
2017-10-22 15:21:29,913 : INFO : PROGRESS: at 46.54% examples, 490891 words/s, i

In [60]:
def makeFeatureVec(words,model,num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float128")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #sorted_set = set(model.sort_vocab())
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [61]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float128")
    # 
    # Loop through the reviews
    for review in reviews:
    # Print a status message every 1000th review
        if counter%1000. == 0.:
            print "Review %d of %d" % (counter, len(reviews))

    # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
    # Increment the counter
        counter = counter + 1.
    return reviewFeatureVecs

In [62]:
DataVecs = getAvgFeatureVecs( sentences, model, num_features )

Review 0 of 10852
Review 1000 of 10852
Review 2000 of 10852
Review 3000 of 10852
Review 4000 of 10852
Review 5000 of 10852
Review 6000 of 10852
Review 7000 of 10852
Review 8000 of 10852
Review 9000 of 10852
Review 10000 of 10852


In [42]:
df_data = pd.DataFrame(DataVecs)
Data = df_data.fillna(lambda x: x.median(),inplace=True)
DataVecs = np.array(Data)

In [63]:
DataVecs.shape

(10852, 100)

In [64]:
print DataVecs[0]

[-0.00075793864  0.0014143679 -0.00064184136 -0.00065346044  0.00056638861
  0.00013978773 -0.0018282468  0.0017588657  0.0001113221  0.00018784285
 -0.00074223369 -0.00066928662  0.00073860228  0.00085501474  0.00016556708
 -9.0846831e-05  0.00070181806  0.00039688861  0.00061576834 -0.0012330097
 -0.0010630462 -0.0010053299  0.00019288843  0.0011641654 -0.00031170713
  0.00017208731 -0.0016305703 -0.0011764182 -0.0001937766  0.001173714
  6.3467034e-05  0.00054574024 -9.1282753e-05 -0.00092442499  0.0013947285
 -0.00021962129  0.00024078352  0.00078113046  0.001029668 -0.00071157415
  0.00062179888 -0.001304497 -0.0016066177  0.0009695506 -0.0021579545
 -0.00040158178 -0.0005142245  0.00036338295  0.0014638985  0.001876543
  0.0014958521 -0.00077078031 -0.0013001869 -0.0012497451 -0.0011449946
  6.7762397e-05  0.0013245822 -0.00024191016 -0.0010481758  0.00096087651
 -0.0020128712  0.00056193481 -0.00082835217  0.00040287977  0.0013398886
  7.3834025e-05 -0.00057429292 -0.00033140801